## IV - Analyses descriptives et Visualisation : Prix de l'immobilier et Risque d'inondation dans le Gard

In [20]:
# Imports des modules nécessaires à l'execution du notebook
import requests
import pandas as pd
import plotly.express as px
import folium
import plotly.io as pio
pio.renderers.default = "notebook"

In [ ]:
!pip install nbformat --upgrade


### Utilisation de L'API indicateurs de prix (accès libre)
L'API Données foncières permet d'interroger rapidement et librement les indicateurs de prix issus de DV3F à différentes échelles géographiques.
Nous appelons l'api pour des premiers graphiques sur la variations des prix à l'échelle départementale. 

In [22]:
BASE_URL_API = "https://apidf-preprod.cerema.fr"

In [23]:
# Fonction d'interrogation de l'API
def apidf(url_endpoint, token=None):
    HEADERS = {
        "Content-Type": "application/json",
    }
    if token:
        HEADERS["Authorization"] = "Token " + token
    response = requests.get(
        url_endpoint,
        headers=HEADERS,
    )  
    if response.status_code == 200:
      return response.json()
    return None

La fonction apidf proposée facilite l'interrogation de l'API Données foncières (en fournissant le jeton si l'accès en restreint)

In [24]:
# Paramétrage du endpoint
code_insee, nom = "30189", "Nîmes"
url = BASE_URL_API + f"/indicateurs/dv3f/communes/annuel/{code_insee}"

# Interrogation de l'API et récupération d'un dataframe
response  = apidf(url)
indicateurs = pd.DataFrame.from_dict(response["results"])

In [ ]:
indicateurs.head()
indicateurs.tail()

In [ ]:
#Filtre des indicateurs propre à notre étude 
df = indicateurs[["valeurfonc_sum_cod1","valeurfonc_sum_cod111","valeurfonc_sum_cod121",
"sbati_sum_cod111","sbati_sum_cod121",
"nbtrans_cod111","nbtrans_cod121",
"valeurfonc_median_cod111","valeurfonc_median_cod121",
"sbati_median_cod111","sbati_median_cod121",
"pxm2_median_apx","pxm2_median_amx","pxm2_median_agx",
"pxm2_median_mpx","pxm2_median_mmx","pxm2_median_mgx",]]
df.head()


### Analyse descriptive du marché du logement à Nîmes

In [ ]:
import nbformat
# Edition du graphique
fig = px.line(indicateurs, 
             x='annee', 
             y=['pxm2_median_mmx', 'pxm2_median_amx'], 
             title = f"Evolution annuelle du prix médian des logements sur {nom}", 
             range_y=[0, 5500],
             labels={"annee" : "Année de mutation", 
                     "value" : "Prix en €/m2",},
             )
noms={"pxm2_median_mmx": "Maison moyenne (entre 90 et 130 m2)", 
      "pxm2_median_amx": "Appartement moyen (T3 et T4)"}
fig.update_layout(legend_title_text="Prix médian au mètre carré")
fig.for_each_trace(lambda t: t.update(hovertemplate = t.hovertemplate.replace(t.name, noms[t.name]), name=noms[t.name]))
fig.show()

In [ ]:
print(nbformat.__version__)

Le prix au mètre carrée reste stable avec une légère hausse en 2023. En prenant en compte l'inflation, le coût de l'immobilier dans le temps diminue. Donc l'immobilier serait plutôt en perte de vitesse. 

In [ ]:
# Paramétrage du endpoint
code_insee, nom = "30189", "Nîmes"
url = BASE_URL_API + f"/indicateurs/dv3f/communes/annuel/{code_insee}"

# Interrogation de l'API et récupération d'un dataframe
response  = apidf(url)
indicateurs = pd.DataFrame.from_dict(response["results"])

# Edition du graphique
fig = px.bar(indicateurs, 
             x='annee', 
             y=['nbtrans_cod111', 'nbtrans_cod121'], 
             title = f"Evolution annuelle du nombre de ventes de logements individuels à {nom}", 
             labels={"annee" : "Année de mutation", 
                     "value" : "Nombre de ventes",},
             )
noms={"nbtrans_cod111": "Maison individuelle", 
      "nbtrans_cod121": "Appartement individuel"}
fig.update_layout(legend_title_text="Nombre de ventes")
fig.for_each_trace(lambda t: t.update(hovertemplate = t.hovertemplate.replace(t.name, noms[t.name]), name=noms[t.name]))
fig.show()

Cependant, le nombre de transactions croît à Nîmes. Une légère baisse en 2014 qui peut s'expliquer par la crise de 2008 à 2012 et les risques climatiques. Il y a par la suite un regain dans le nombre de transactions immobilières. Cela peut à certains égards contredire l'idée selon laquelle les risques climatiques ralentissent l'immobilier dans ces régions. 
Peut-être que Nîmes est moins touchés par ce phénomène.

Nous analysons donc Saint-Gilles, une commune au sud de Nîmes qui est plus sujette au risque d'inondation. 

In [ ]:
# Paramétrage du endpoint
code_insee, nom = "30258", "Saint-Gilles"
url = BASE_URL_API + f"/indicateurs/dv3f/communes/annuel/{code_insee}"

# Interrogation de l'API et récupération d'un dataframe
response  = apidf(url)
indicateurs = pd.DataFrame.from_dict(response["results"])

# Edition du graphique
fig = px.bar(indicateurs, 
             x='annee', 
             y=['nbtrans_cod111', 'nbtrans_cod121'], 
             title = f"Evolution annuelle du nombre de ventes de logements individuels à {nom}", 
             labels={"annee" : "Année de mutation", 
                     "value" : "Nombre de ventes",},
             )
noms={"nbtrans_cod111": "Maison individuelle", 
      "nbtrans_cod121": "Appartement individuel"}
fig.update_layout(legend_title_text="Nombre de ventes")
fig.for_each_trace(lambda t: t.update(hovertemplate = t.hovertemplate.replace(t.name, noms[t.name]), name=noms[t.name]))
fig.show()

Saint-Gilles connaît des variations similaires que Nîmes. Le risques climatiques ne ralentirait pas le nombre de transactions immobilière, en tout cas, cela a moins d'effet qu'une crise économique sur le marché immobilier. 

In [ ]:
# Paramétrage du endpoint
code_insee, nom = "30189", "Nîmes"
url = BASE_URL_API + f"/dvf_opendata/mutations/?code_insee={code_insee}&page_size=1000&anneemut_min=2018&codtypbien=111"

# Interrogation de l'API pour récupérer les pages de mutations
pages = []

while True:
    response = apidf(url) 
    mutations = pd.DataFrame.from_dict(response["results"])
    pages.append(mutations)
    if not response["next"]:
      break
    url = response["next"]

# concaténation des pages et affichage graphique
mutations = pd.concat(pages)
mutations["valeurfonc"] = mutations["valeurfonc"].astype(float)
fig = px.violin(mutations, 
                y="valeurfonc", 
                x="anneemut", 
                color="anneemut", 
                box=True, 
                title = f"Distribution annuelle des prix des ventes de maison à partir de 2018 à {nom}", 
                labels={"annee" : "Année", 
                        "valeurfonc" : "Prix en €",},)
fig.update_layout(legend_title_text="Année de mutation")
fig.show()

La distribution des logements, qui est similaire à travers les années, montre qu'il n'y a pas de différence flagrante sur le marché de l'immobilier malgré l'augmentation du risque dans ces régions. 

## Analyse descriptive du marché du logement dans le Gard 
### A- Valeur médiane par commune dans le Gard

Nous créons ici une nouvelle dataframe qui collecte toutes les valeurs foncières médiane des communes du Gard.

D'abord nous récupérons, grâce au fichiers de l'Insee, tous les codes communes du Gard.

In [ ]:
# URL for the INSEE commune data
url = "https://www.insee.fr/fr/statistiques/fichier/6800675/v_commune_2023.csv"

# Load the CSV file containing the list of communes
communes = pd.read_csv(url)
print("File successfully loaded!")
communes.head()




In [ ]:
# Filtrer les communes du Gard (département 30)
gard_communes = communes[communes["DEP"] == "30"]

# Extraire les codes INSEE
codes_communes = gard_communes["COM"].tolist()

# Diviser la liste des communes en sous-listes de taille 35
chunk_size = 35
sub_lists = [codes_communes[i:i + chunk_size] for i in range(0, len(codes_communes), chunk_size)]

print(f"Nombre total de communes dans le Gard : {len(codes_communes)}")
print(f"Nombre de sous-listes : {len(sub_lists)}")



Nous avons créer des sous listes des 350 codes communes pour alléger le code suivant car l'api s'arrêtait après trop de requête de suite. 

**Ce code ne doit pas être lancé car il dure environ 30 minutes et les communes se chargent de manière aléatoire au bon vouloir de l'api.** 

In [ ]:

import time

# Initialisation d'une liste pour stocker les DataFrames
dataframes = []

for sub_list in sub_lists:
    for code_insee in sub_list:
        url = BASE_URL_API + f"/indicateurs/dv3f/communes/annuel/{code_insee}"

        for attempt in range(3):  # Essayer jusqu'à 3 fois
            try:
                print(f"Requête pour {code_insee}: {url}")
                response = apidf(url)
                break
            except ConnectionResetError as e:
                print(f"Erreur de connexion pour {code_insee}: {e}")
                time.sleep(3)
            except Exception as e:
                print(f"Autre erreur : {e}")
                break
        else:
            print(f"Échec après 3 tentatives pour {code_insee}")
            continue

        # Traiter les données en cas de succès
        indicateurs_communes = pd.DataFrame.from_dict(response.get("results", []))
        if not indicateurs_communes.empty:
            val_fonc = indicateurs_communes[indicateurs_communes['annee'] == '2023']
            val_fonc_2023 = val_fonc[["codgeo", "libgeo", "valeurfonc_median_cod111", "valeurfonc_median_cod121"]]
            dataframes.append(val_fonc_2023)

    time.sleep(2)  # Pause entre les groupes pour éviter la surcharge

# Combiner les DataFrames
final_dataframe = pd.concat(dataframes, ignore_index=True) if dataframes else pd.DataFrame()

# Afficher le DataFrame final
final_dataframe.head()




Nous avons donc créer une nouvelle dataframe qui se concentre sur les valeurs foncières de 2023. Elle est enregistré en csv pour faciliter les codes suivants. 

In [ ]:
final_dataframe.to_csv("/home/onyxia/work/Python-For-Data-Science-Project/final_dataframe.csv", index=False, encoding="utf-8")


### B- Création d'une carte des prix dans le Gard

Nous cherchons maintenant à créer une carte du Gard qui rendra compte des valeurs fonciers de chaque communes en 2023. 

In [ ]:
# Importation du package Cartiflette 
!pip install py7zr geopandas openpyxl tqdm s3fs
!pip install PyYAML xlrd
!pip install git+https://github.com/inseefrlab/cartiflette

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from cartiflette import carti_download



communes_30= carti_download(
    values=["30"],
    crs=4326,
    borders="COMMUNE",
    vectorfile_format="geojson",
    filter_by="DEPARTEMENT",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022,
)

communes_30.crs
communes_30 = communes_30.to_crs(2154)
communes_30.crs

ax = communes_30.boundary.plot()
ax.set_axis_off()



Nous avons créer grâce à cartiflette le fond de carte du Gard. On va maintenant y ajouter la base de données des valeurs foncières médianes de 2023. 

In [ ]:
# Charger le fichier CSV des valeurs foncières
final_dataframe = pd.read_csv("final_dataframe.csv", encoding="utf-8")
print(final_dataframe.head())


In [ ]:
print(communes_30["INSEE_COM"].dtype)  # Vérifier le type de INSEE_COM
print(final_dataframe["codgeo"].dtype)  # Vérifier le type de codgeo

Les types ne correspondant pas, on les change en chaines de caractères. 

In [14]:
# Convertir les colonnes en chaînes de caractères
communes_30["INSEE_COM"] = communes_30["INSEE_COM"].astype(str)
final_dataframe["codgeo"] = final_dataframe["codgeo"].astype(str)


On peut maintenant concaténer les deux bases de données.

In [ ]:
# Fusionner les données géographiques avec les données foncières
communes_30 = communes_30.merge(final_dataframe, how="left", left_on="INSEE_COM", right_on="codgeo")

print(communes_30.head())


On trace maintenant la carte des prix du Gard pour les maisons individuelles. 

In [ ]:
# Afficher la carte avec une coloration en fonction des valeurs foncières
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
communes_30.plot(column="valeurfonc_median_cod111",  # Colonne avec les valeurs foncières
                 cmap="OrRd",                      # Colormap (nuances de rouge)
                 legend=True,                      # Ajouter une légende
                 ax=ax)

ax.set_title("Carte des valeurs foncières des maisons des communes du Gard (2023)", fontsize=16)
ax.set_axis_off()  # Enlever les axes
plt.show()


La carte étant un peu vide avec certaine zone vide, on décide de griser les zones sans informations et de faire la moyenne des valeurs foncières des appartements et des maisons pour compléter la carte. 

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

# Étape 1 : Calculer la moyenne des deux colonnes
communes_30["valeurfonc_moyenne"] = (
    communes_30[["valeurfonc_median_cod111", "valeurfonc_median_cod121"]]
    .mean(axis=1, skipna=True)
)

# Étape 2 : Ajouter une colonne pour marquer les communes sans informations
communes_30["data_missing"] = communes_30["valeurfonc_moyenne"].isna()

# Étape 3 : Définir les couleurs
# Palette de couleurs : communes avec données (bleu -> rouge) et sans données (gris)
color_map = communes_30["valeurfonc_moyenne"].apply(
    lambda x: "gray" if np.isnan(x) else plt.cm.viridis((x - communes_30["valeurfonc_moyenne"].min()) / 
                                                        (communes_30["valeurfonc_moyenne"].max() - communes_30["valeurfonc_moyenne"].min()))
)

# Étape 4 : Tracer la carte
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
communes_30.plot(ax=ax, color=color_map)

# Ajouter une légende
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D

# Couleur spéciale pour les données manquantes
missing_patch = Line2D(
    [0], [0], color="gray", lw=4, label="Données manquantes"
)

# Ajouter des légendes automatiques pour les autres valeurs
sm = plt.cm.ScalarMappable(cmap="viridis", norm=plt.Normalize(vmin=communes_30["valeurfonc_moyenne"].min(), vmax=communes_30["valeurfonc_moyenne"].max()))
sm._A = []
cbar = fig.colorbar(sm, ax=ax, orientation="vertical", fraction=0.03, pad=0.04)
cbar.set_label("Valeurs foncières moyennes")

ax.legend(handles=[missing_patch], loc="lower right")
ax.set_title("Carte des valeurs foncières moyennes par commune (Gard)")
ax.axis("off")

# Afficher la carte
plt.show()


Sans prendre en compte le nord du Gard qui doit être moins attractif, les zones les plus touchés par les risques d'inondations qui sont au sud du Gard(vu sur la carte des risques) ont des valeurs foncières médianes moindre, ce qui traduirait une moindre attractivité de ces zones à risques en 2023.

## Risque d'inondation et transactions immobilières 

L'objectif de cette partie est de coupler les données récupéré au travers du processing des différentes sources de données afin de représenter dans un même temps risque d'innondation et transactions immobilières 

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from cartiflette import carti_download
import matplotlib.patches as mpatches

#Récupération des geodataframes pour tracer la carte 
gdf_nv_risque = gpd.read_file("/home/onyxia/work/Python-For-Data-Science-Project/II - TRI Shapefile Processing/gdf_nv_risque.geojson")
gdf_type_risque = risk_data = pd.read_csv("/home/onyxia/work/Python-For-Data-Science-Project/I - TRI API Processing /Type_inondation_data.csv")
gdf_cours_eau = gpd.read_file("/home/onyxia/work/Python-For-Data-Science-Project/II - TRI Shapefile Processing/gdf_cours_eau.geojson")

# Convertir les colonnes de type Timestamp en chaînes de caractères dans gdf_nv_risque
for col in gdf_nv_risque.columns:
    if gdf_nv_risque[col].dtype == 'datetime64[ms]':
        gdf_nv_risque[col] = gdf_nv_risque[col].astype(str)


# Charger les données Cartiflette pour les communes du Gard
Gard = carti_download(
    crs=4326,
    values="30",
    borders="COMMUNE",
    vectorfile_format="geojson",
    filter_by="DEPARTEMENT",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022
)

# Charger les informations sur la nature des risques
gdf_type_risque_grouped = gdf_type_risque.groupby('code_insee')['libelle_risque_long'].apply(
    lambda x: ', '.join(x.unique())
).reset_index()
Gard_type_risque = Gard.merge(gdf_type_risque_grouped, left_on='INSEE_COM', right_on='code_insee', how='left')
Gard_type_risque['libelle_risque_long'] = Gard_type_risque['libelle_risque_long'].fillna("Pas de risque identifié")


In [ ]:
import folium
from folium.features import GeoJson, GeoJsonTooltip

# Créer une carte Folium centrée sur le Gard
m = folium.Map(location=[44.0, 4.0], zoom_start=9, tiles="cartodbpositron")

# Ajouter les contours des communes et la nature des risques avec `Gard_with_risks`
geojson_communes = folium.GeoJson(
    Gard_type_risque,
    style_function=lambda x: {"fillColor": "transparent", "color": "black", "weight": 1, "fillOpacity": 0},
    highlight_function=lambda x: {"fillColor": "yellow", "color": "black", "weight": 2, "fillOpacity": 0.3},
    tooltip=GeoJsonTooltip(
        fields=["libelle_risque_long"],  # Afficher la nature des risques
        aliases=["Nature du risque :"],  # Texte dans l'info-bulle
        localize=True
    )
)
geojson_communes.add_to(m)

#Afficher la carte
m


In [ ]:
import folium
from folium.features import GeoJson, GeoJsonTooltip

# Créer une carte Folium centrée sur le Gard
m = folium.Map(location=[44.0, 4.0], zoom_start=9, tiles="cartodbpositron")

# Étape 1 : Ajouter les contours des communes et la nature des risques avec `Gard_with_risks`
geojson_communes = folium.GeoJson(
    Gard_type_risque,
    style_function=lambda x: {"fillColor": "transparent", "color": "black", "weight": 1, "fillOpacity": 0},
    highlight_function=lambda x: {"fillColor": "yellow", "color": "black", "weight": 2, "fillOpacity": 0.3},
    tooltip=GeoJsonTooltip(
        fields=["libelle_risque_long"],  # Afficher la nature des risques
        aliases=["Nature du risque :"],  # Texte dans l'info-bulle
        localize=True
    )
)
geojson_communes.add_to(m)

# Étape 2 : Ajouter les cours d’eau dans le Gard avec `waterways_clipped`
geojson_waterways = folium.GeoJson(
    gdf_cours_eau,
    style_function=lambda x: {"color": "blue", "weight": 1.5, "fillOpacity": 0.7},  # Style des cours d'eau
    tooltip=GeoJsonTooltip(fields=[], aliases=[])  # Pas d'info-bulle pour les cours d'eau
)
geojson_waterways.add_to(m)

#afficher la carte
m


In [ ]:
import folium
from folium.features import GeoJson, GeoJsonTooltip

# Créer une carte Folium centrée sur le Gard
m = folium.Map(location=[44.0, 4.0], zoom_start=9, tiles="cartodbpositron")

# Étape 1 : Ajouter les polygones des niveaux de risque en arrière-plan (`gdf_nv_risque`)
def style_function_risk(feature):
    risk_level = feature['properties'].get('typ_inond1', None)
    if risk_level == 1:  # Risque faible
        color = "lightblue"
    elif risk_level == 3:  # Risque élevé
        color = "lightcoral"
    else:
        color = "gray"  # Pas de données
    return {"fillColor": color, "color": "black", "weight": 0.5, "fillOpacity": 0.6}

geojson_risks = folium.GeoJson(
    gdf_nv_risque,
    style_function=style_function_risk  # Appliquer les couleurs basées sur `typ_inond1`
)
geojson_risks.add_to(m)

# Étape 2 : Ajouter les contours des communes avec surlignage et info-bulle (`Gard_with_risks`)
geojson_communes = folium.GeoJson(
    Gard_type_risque,
    style_function=lambda x: {"fillColor": "transparent", "color": "black", "weight": 1, "fillOpacity": 0},
    highlight_function=lambda x: {"fillColor": "yellow", "color": "black", "weight": 2, "fillOpacity": 0.3},
    tooltip=GeoJsonTooltip(
        fields=["libelle_risque_long"],  # Afficher la nature des risques
        aliases=["Nature du risque :"],  # Texte dans l'info-bulle
        localize=True
    )
)
geojson_communes.add_to(m)

# Étape 3 : Ajouter les cours d’eau dans le Gard avec `waterways_clipped`
geojson_waterways = folium.GeoJson(
    gdf_cours_eau,
    style_function=lambda x: {"color": "blue", "weight": 1.5, "fillOpacity": 0.7}
)
geojson_waterways.add_to(m)

# Étape 4 : Ajouter une légende
legend_html = """
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 250px;
    background-color: white;
    z-index:9999;
    padding: 10px;
    border: 1px solid gray;
">
    <h4>Légende</h4>
    <i style="background: lightblue; width: 15px; height: 15px; float: left; margin-right: 5px;"></i> Risque faible<br>
    <i style="background: lightgreen; width: 15px; height: 15px; float: left; margin-right: 5px;"></i> Risque modéré<br>
    <i style="background: lightcoral; width: 15px; height: 15px; float: left; margin-right: 5px;"></i> Risque élevé<br>
    <i style="border: 1px solid black; width: 15px; height: 15px; float: left; margin-right: 5px;"></i> Bordure des communes<br>
    <i style="background: blue; width: 15px; height: 15px; float: left; margin-right: 5px;"></i> Cours d'eau<br>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

#afficher la carte
m


In [ ]:
# Nous cherchons à évaluer l'interconnection entre les deux sources de données sur les inondations 

gdf_nv_risque_projected = gdf_nv_risque.to_crs(epsg=2154)  # CRS projeté
Gard_type_risque_projected = Gard_type_risque.to_crs(epsg=2154)

#Filtrer les zones avec une nature de risque déterminée
Gard_with_valid_risks = Gard_type_risque_projected[Gard_type_risque_projected['libelle_risque_long'] != "Pas de risque identifié"]

#la fonction area permet de déterminer la superficie totale des zones à risque d'inondation dans le Gard. 
gdf_nv_risque_projected['area'] = gdf_nv_risque_projected.geometry.area
total_area = gdf_nv_risque_projected['area'].sum()

#Jointure spaciale afin de déterminer les zones de chevauchement entre nature et niveau de risque 
intersections = gpd.overlay(Gard_with_valid_risks, gdf_nv_risque_projected, how="intersection")

#déterminer la superficie totale des intersections
intersections['area'] = intersections.geometry.area
intersected_area = intersections['area'].sum()

#Finalement, nous calculons le le pourcentage
percentage_intersection = (intersected_area / total_area) * 100

print(f"Le Pourcentage d'intersection entre territoire avec un niveau de risque supérieur ou égal à un et un libellé de risque est de {percentage_intersection:.2f}%")


## Visualisation : transactions immobilières et Risque d'inondation 

In [ ]:
import folium
from folium.features import GeoJson, GeoJsonTooltip
from folium.plugins import MarkerCluster
import geopandas as gpd

gdf_trans_immo = gpd.read_file('/home/onyxia/work/Python-For-Data-Science-Project/III - DVF Processing/gdf_trans_immo.geojson')

# Créer une carte Folium centrée sur le Gard
m = folium.Map(location=[44.0, 4.0], zoom_start=9, tiles="cartodbpositron")

# Étape 1 : Ajouter les polygones des niveaux de risque (`gdf_nv_risque`)
def style_function_risk(feature):
    risk_level = feature['properties'].get('typ_inond1', None)
    if risk_level == 1:  # Risque faible
        color = "lightblue"
    elif risk_level == 3:  # Risque élevé
        color = "lightcoral"
    else:
        color = "gray"  # Pas de données
    return {"fillColor": color, "color": "black", "weight": 0.5, "fillOpacity": 0.6}

geojson_risks = folium.GeoJson(
    gdf_nv_risque,
    style_function=style_function_risk
)
geojson_risks.add_to(m)

# Étape 2 : Ajouter les contours des communes (`Gard_with_risks`)
geojson_communes = folium.GeoJson(
    Gard_type_risque,
    style_function=lambda x: {"fillColor": "transparent", "color": "black", "weight": 1, "fillOpacity": 0},
    )

geojson_communes.add_to(m)

# Étape 3 : Ajouter les cours d’eau dans le Gard (`waterways_clipped`)
geojson_waterways = folium.GeoJson(
    gdf_cours_eau,
    style_function=lambda x: {"color": "blue", "weight": 1.5, "fillOpacity": 0.7}
)
geojson_waterways.add_to(m)

# Étape 4 : Ajouter les points des transactions immobilières (`gdf_trans_immo`) avec un MarkerCluster
marker_cluster = MarkerCluster().add_to(m)

for _, row in gdf_trans_immo.iterrows():
    # Vérifier que la géométrie est un Point
    if row.geometry.geom_type == "Point":
        # Extraire les informations pour la tooltip
        valeurfonc = row.get("properties.valeurfonc", "Non disponible")
        datemut = row.get("properties.datemut", "Non disponible")
        sbati = row.get("properties.sbati", "Non disponible")
        libtypbien = row.get("properties.libtypbien", "Non disponible")

        # Ajouter le marqueur avec la tooltip
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            tooltip=(
                f"<b>Valeur foncière :</b> {valeurfonc}<br>"
                f"<b>Date de transaction :</b> {datemut}<br>"
                f"<b>Nombre de m² :</b> {sbati}<br>"
                f"<b>Type de propriété :</b> {libtypbien}"
            )
        ).add_to(marker_cluster)

# Étape 5 : Ajouter une légende
legend_html = """
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 250px;
    background-color: white;
    z-index:9999;
    padding: 10px;
    border: 1px solid gray;
">
    <h4>Légende</h4>
    <i style="background: lightblue; width: 15px; height: 15px; float: left; margin-right: 5px;"></i> Risque faible<br>
    <i style="background: lightcoral; width: 15px; height: 15px; float: left; margin-right: 5px;"></i> Risque élevé<br>
    <i style="border: 1px solid black; width: 15px; height: 15px; float: left; margin-right: 5px;"></i> Bordure des communes<br>
    <i style="background: blue; width: 15px; height: 15px; float: left; margin-right: 5px;"></i> Cours d'eau<br>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

#Afficher la carte
print("Carte combinée générée et sauvegardée dans 'carte_transaction_risque.html'.")
m

L'étape suivante consiste en la régression linéaire à partir du fichier géospatial des transactions immobilières et celles des zones à risques dans le Gard : 

Pour ce faire, il était souhaitable d'ajouter le niveau de risque présent dans la geodataframe 'gdf_nv_risque' à la géodataframe 'gdf_trans_immo' contenant les valorisation foncières et leurs emplacement géographique pour marquer le niveau de risque d'inondation à chaque transaction. Cependant, nous n'avons pas réussi à réaliser la jointure spatiale de ces deux geodataframe après de multiples tentatives que vous trouverez ci dessous : 

**code tentative 1 : avec la fonction sjoin**

import pandas as pd

import geopandas as gpd

import json

from shapely.geometry import shape


Charger les données des transactions

df = pd.read_csv('/home/onyxia/work/Python-For-Data-Science-Project/DVF Processing/transactions_dvf_gard_filtre.zip')

Convertir les chaînes de caractères en listes JSON

df["geometry.coordinates"] = df["geometry.coordinates"].apply(json.loads)  # Ne pas exécuter plusieurs fois si déjà converti

Créer des objets géométriques Shapely

df["geometry"] = df["geometry.coordinates"].apply(
    lambda coords: shape({"type": "MultiPolygon", "coordinates": coords})
)

gdf_immo = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:2154")  # CRS projeté pour calculs précis

Harmoniser les systèmes de coordonnées

if gdf_immo.crs != gdf_nv_risque.crs:

    gdf_nv_risque = gdf_nv_risque.to_crs(gdf_immo.crs)

Nettoyer les géométries

gdf_immo = gdf_immo[gdf_immo.geometry.notnull()]

gdf_nv_risque = gdf_nv_risque[gdf_nv_risque.geometry.notnull()]

gdf_nv_risque['geometry'] = gdf_nv_risque.geometry.buffer(0)  # Réparer les géométries si nécessaire

Effectuer la jointure spatiale

gdf_with_risk = gpd.sjoin(
    gdf_immo,                         # Points des transactions
    gdf_nv_risque[['geometry', 'typ_inond1']],  # Polygones avec niveaux de risque
    how="left",                             # Conserve toutes les transactions
    predicate="within"                      # Associe les points contenus dans les polygones
)


Ajouter une valeur par défaut pour les transactions hors zones de risque

gdf_with_risk['typ_inond1'] = gdf_with_risk['typ_inond1'].fillna(0)

Renommer la colonne pour clarifier

gdf_with_risk = gdf_with_risk.rename(columns={"typ_inond1": "niveau_risque_inondation"})

Supprimer les colonnes inutiles

gdf_with_risk = gdf_with_risk.drop(columns=["index_right"], errors="ignore")

Afficher un aperçu des données jointes

print(gdf_with_risk.head())

Sauvegarder le résultat

gdf_with_risk.to_file("transactions_avec_risque.geojson", driver="GeoJSON")

print("GeoDataFrame sauvegardée dans 'transactions_avec_risque.geojson'.")

**code tentative 2 : avec la fonction shapely contains** 

Dans ce code, nous avions pu identifier que l'erreur pouvais consister au fait que les geodata étaient exprimés d'un coté par des (multi)polygon et de l'autre par des points, nous sommes donc revenu au fichier initial de transactions immobilières (ici df) dont les geodata étaient encodées sous forme de (multi)polygon. 

import pandas as pd

import geopandas as gpd

import json

from shapely.geometry import shape

Charger les données des transactions

df = pd.read_csv('/home/onyxia/work/Python-For-Data-Science-Project/DVF Processing/transactions_dvf_gard_filtre.zip')

Convertir les chaînes de caractères en listes JSON

df["geometry.coordinates"] = df["geometry.coordinates"].apply(json.loads)  # Ne pas exécuter plusieurs fois si déjà converti

Créer des objets géométriques Shapely

df["geometry"] = df["geometry.coordinates"].apply(
    lambda coords: shape({"type": "MultiPolygon", "coordinates": coords})
)

gdf_immo = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:2154")  # CRS projeté pour calculs précis

Harmoniser les systèmes de coordonnées

if gdf_immo.crs != gdf_nv_risque.crs:

    gdf_nv_risque = gdf_nv_risque.to_crs(gdf_immo.crs)

Nettoyer les géométries

gdf_immo = gdf_immo[gdf_immo.geometry.notnull()]

gdf_nv_risque = gdf_nv_risque[gdf_nv_risque.geometry.notnull()]

Réparer les géométries si nécessaire

gdf_nv_risque['geometry'] = gdf_nv_risque.geometry.buffer(0)

Ajouter une colonne pour stocker le niveau de risque

def get_risk_level(point, risk_polygons):
    """
    Retourne le niveau de risque pour un point donné en vérifiant son inclusion dans les polygones de risque.
    """
    for _, row in risk_polygons.iterrows():
        if row.geometry.contains(point):
            return row['typ_inond1']  # Retourner le niveau de risque si le point est contenu dans le polygone
    return 0  # Aucun risque si le point n'est dans aucun polygone

Calculer le niveau de risque pour chaque transaction

gdf_immo['niveau_risque_inondation'] = gdf_immo.geometry.apply(lambda point: get_risk_level(point, gdf_nv_risque))

Afficher un aperçu des données jointes

print(gdf_immo[['geometry', 'niveau_risque_inondation']].head())

Sauvegarder le résultat si nécessaire

gdf_immo.to_file("transactions_avec_risque.geojson", driver="GeoJSON")

print("GeoDataFrame sauvegardée dans 'transactions_avec_risque.geojson'.")

Ainsi, la tentative de mise en évidence de la décote lié au niveau de risque d'inondation dans le Gard a été soldée par un échec, comme nous n'avons pas réussi à coupler à temps le niveau de risque d'inondation aux transactions immobilières. 
Nous avons quand bien même tenté de vous signifier notre démarche scientifique dans la forme qu'elle comptait prendre ci dessous. 


# Modélisation quantitative

Cette section vient compléter l'analyse exploratoire menée plus haut pour essayer d'offrir une modeste quantification de la relation entre risque d'inondations et prix de l'immobilier, sans essayer d'obtenir une interprétation qui soit causale. 

Pour ce faire, nous nous reposons sur un regroupement de données cross-sectionnelles (*pooled cross-sectional* en anglais) que nous avons pu assembler à partir des bases de données DVF et Géorisque. 
Ces données comprennent notamment les variables suivantes:
- La valeur à laquelle s'est déroulée la transaction immobilière, que nous ajusterons de l'inflation.
- Le niveau de risque inhérent à la position géographique du bien, qui est encodé entre trois niveaux de risques: faible, moyen et élevé.
- La surface en mètre carrée du bien.

Nous estimerons ainsi l'équation économétrique suivante:

$ Prix = \beta_0 + \beta_1*R_{élevé} + \beta_2*R_{moyen} + \beta_3*Surface + \epsilon $

Avec:
- $Prix$, la valeur de la transaction immobilière en euro constant de 2015. Nous faisons cet ajustement pour isoler l'effet de l'inflation et donc mieux capturer les interactions liées aux risques d'inondation. De plus, nous avons exclu de la base de données les observations où les valeurs de transactions étaient supérieures au 95ème quantile ou inférieure au 5ème quantile. Ceci permet de se séparer des valeurs aberrantes qui pourraient avoir un effet disproportionné sur nos coefficients de régression.
- $R_{élevé}$ et $R_{moyen}$ sont nos variables binaires (*dummies*) liés au niveau de risque du bien immobilier. Le niveau de référence pour de ces *dummies* est le risque de niveau faible.
- Et pour finir, $Surface$ correspond à la surface en mètre carré du bien immobilier. 


cette modélisation fait également référence au prix en euro constant, que nous souhaitions capturer afin de prendre en compte l'inflation dans l'évolution des prix, que nous souhaitions calculer à partir du notebook `Calcul des prix en euro constant.ipynb` mis en annexe